# NO2 Detection Application 

In [1]:
Name =  "Evan Gallagher"

### Abstract
Nitrogen Dioxide emissions have been on the increase for nearly 50 years, primarily caused by combustion of fossil fuels and energy derived from coal power plants. Although nitrogen oxide emissions are expected to decrease in countries with increased electric transportation, "rapid economic development has the potential to increase [NO2 emissions] significantly." Specifically, nitrogen oxide concentrations have seen an "increase of about 50%... over industrial areas of China." (Richter) An application that could inform someone about surrounding NO2 concentrations shows potential to both keep the public informed and potentially save lives. 

## NO2 Emissions over China: 
![alt text](https://www.gpsworld.com/wp-content/uploads/2020/03/ESA-NO2-Copernicus-video-screenshot-COVID-19-W.jpg "Logo Title Text 1")
Sentinel-5P Nitrogen dioxide data over industrial China, 2019. Data collected from Descartes Labs: https://www.gpsworld.com/analysis-of-satellite-imagery-shows-reduced-no2-in-china-italy/

## The Goal: 
* To create an application that can create an accurate reading of surrounding NO2 concentration given a location. 



## Importance: 
According to the European Respiratory Journal, "short-term NO2 exposure at about 500 micrograms.m3 affects human bronchial responsiveness."(Bylin) This study was done with merely 30 minutes of exposure, which draws attention to the above tropospheric NO2 graph of industrial China. Particularly in large cities such as Beijing and Hong Kong, NO2 is averaging at potentially harmful levels. China is far from the only country to be experiencing this, from India to South Africa, many countries are seeing a rise in NO2 concentration. This is to say that an application that can take any given location, and provide an accurate reading of NO2 would be broadly applicable around the world. 



## Data 

* `Tropomi:` Tropomi data is taken by Sentinel-5P which has an on board sensor for tropospheric NO2. It is constantly taking data, and produces a near real time feed of the NO2 data. 
* `Global Power Plant Database:` a dataset provided by the World Resources Institute. This is useful to help decide what if recorded NO2 is merely a product of large city fossil fuel combustion, or a local power plant
* `ERA5 Daily aggregates:` ERA5 provides aggregated values each day for seven ERA5 climate reanalysis parameters. We are particularly interested bands 8 and 9 which can be used to calculate wind direction and speed at a given pixel


## Challenges
1. The most immediate challenge comes from the NO2 Tropomi data itself. Pixels for Tropomi data cover about 5 km for 1 NO2 value. If we want to look at the NO2 values surrounding a coal plant, we have a limited amount of pixels for the surrounding area to go off of. 
2. There are some discrepancies in what we observe from this data as well. While large urban areas tend to reflect NO2 levels accurately, power plants do not. Spikes in NO2 often appear multiple pixels away from known power plants. It is possible that the emissions from these power plants could be more susceptible to wind, which would be something to consider if making the most accurate NO2 detection application. 
3. The final challenge is to integrate wind with Tropomi data. This is where ERA5 comes in. Using the 'u' and 'v' bands from this dataset allow us to find out both wind speed and wind direction at any given pixel. This could be used in combination with Tropomi data to create more accurate NO2 results 

# Study Area
We know that the NO2 concentration of urban areas seems less affected by wind than that generated by power plants. If we want to create an accurate calculation of how wind affects NO2, we can work backwards from our final Tropomi pixel data point. Putting the Tropomi values through the lens of ERA5 wind data, we can try and calculate a value closer to the source than what Tropomi tells us. Picking a single point to test this, I picked the Pawnee Generating Station in Brush, CO because it is both close to home, and produces some interesting Tropomi results.

### Greater Denver Area
![alt text](https://i.ibb.co/KW4F8gb/studyarea-unzoom.png)
A zoomed out version of study area. The yellow pin shows the Pawnee Generating Station 

### Pawnee Generating Plant Study Area
![alt text](https://i.ibb.co/jJBhVdk/Screen-Shot-2020-05-06-at-11-02-56-AM.png "Logo Title Text 1")
Zoomed in study area. The green outline displays the entire area of tropomi data considered. 

**Important note:** This project was done in Google Earth Engine, which uses the Javascript format of coding. Because of this, I display results as screenshots, and will add the corresponding Javascript code later. Google Earth Engine allows for exporting of geotiff files, but I was not able to read them into this notebook properly so I commented out the attempted code at the end. 


# Workflow 
To complete this application, there will three phases: 

* `Phase 1:` Work entirely with one study area to see if I can create a calculation that takes odd Tropomi data points, and returns a value closer to a near power plant. 
* `Phase 2:` Take the "working" calculation, and apply it to other Tropomi data points that seem off. If the calculation does not apply well to other locations, then it will likely need to be reworked until results are satisfactory. 
* `Phase 3:` Take the now broadly-applicable calculation, and create an application that can take an input gps location, and derive a more accurate NO2 calculation. 

# Javascript Code

**Creating Study Area**

`var coal_plant = geometry.buffer(20000)
var polygon = ee.Geometry.Polygon([
  [[-104.3343153554145, 39.97221810099358],
  [-103.29336076557075, 39.97221810099358],
  [-103.29336076557075, 40.4535747006502],
  [-104.3343153554145, 40.4535747006502],
  [-104.3343153554145, 39.97221810099358]]]);`

`Map.addLayer(polygon, {color: "green"}, "Region")`


**Adding in Tropomi**

`Tropomi No2 Data 
var collection = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2')
  .select('NO2_column_number_density')
  .filterDate('2019-06-01', '2019-06-06');`

`var band_viz = {
  min: 0,
  max: 0.0002,
  palette: ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
};`

`var tropomi_mean = collection.mean()`

**Cropping Tropomi to study area** 

`var tropomi_co = ee.Image(tropomi_mean)
  .clip(polygon)`
  

**Adding all created layers to map**

`Map.addLayer(tropomi_co, band_viz, 'S5P N02');
Map.centerObject(polygon);
Map.addLayer(coal_plant, {color: "yellow"}, "Coal")`


**Adding WRI Powerplant database for comparison** 

`var table = ee.FeatureCollection("WRI/GPPD/power_plants");`

`// Get a color from a fuel
var fuelColor = ee.Dictionary({
  'Coal': '000000',
  'Oil': '593704',
  'Gas': 'BC80BD',
  'Hydro': '0565A6',
  'Nuclear': 'E31A1C',
  'Solar': 'FF7F00',
  'Waste': '6A3D9A',
  'Wind': '5CA2D1',
  'Geothermal': 'FDBF6F',
  'Biomass': '229A00'
});`

`// List of fuels to add to the map
var fuels = ['Coal', 'Oil', 'Gas', 'Hydro', 'Nuclear', 'Solar', 'Waste',
    'Wind', 'Geothermal', 'Biomass'];
`

`function addStyle(pt) {
  var size = ee.Number(pt.get('capacitymw')).sqrt().divide(10).add(2);
  var color = fuelColor.get(pt.get('fuel1'));
  return pt.set('styleProperty', ee.Dictionary({'pointSize': size, 'color': color}));
}`

`// Make a FeatureCollection out of the power plant data table
var pp = ee.FeatureCollection(table).map(addStyle);
print(pp.first());`

`function addLayer(fuel) {
  print(fuel);
  Map.addLayer(pp.filter(ee.Filter.eq('fuel1', fuel)).style({styleProperty: 'styleProperty', neighborhood: 50}), {}, fuel, true, 0.65);
}`

`// Apply 'addLayer' to each record in 'fuels'
fuelColor.keys().getInfo().map(addLayer);`

This code adds a few layers to the map. The first is the variable coal_plant which is the coordinates for the Pawnee coal plant, and I extended the area 20 km for scale. The second a custom polygon that I found to have the most interesting tropomi data. Next, I added the tropomi data in at a world wide scale, which I then clipped to the study area in a new variable called "tropomi_co." These all combine to create a nice view of the study area and point out the discrepancies of the Tropomi data. 

## What do we see at the Pawnee Coal plant? 

![alt text](https://i.ibb.co/JHMW2N6/Screen-Shot-2020-05-06-at-11-20-47-AM.png)
Plot of the study area, with relevant Tropomi data laid over. The most interesting results from this are the spikes of NO2 to Northeast and Northwest of Pawnee Generating Station(indicated by black circles), but lower values in its own radius. 

## Results 
We know for certain that the coal production coming from Brush, CO fits the discrepancy we initially were tasked of finding. The Tropomi results do not make sense, as there are two spikes in values of NO2 in our study area, neither of which fit into a 20 km radius of the coal plant. This leads us to believe that there very well could be wind interference with what we see from the Tropomi data. 

I still have not gotten a working calculation using wind data, so this remains the primary objective in moving forward with this project. 

## Conclusions 
This project is far from over. In terms of my workflow overview, I am still in phase 1, however, this is where the bulk of this project is. I still have a lot to learn about the ERA5 wind dataset, as the 'u' and the 'v' bands together can calculate speed and direction. Online sources are conflicting, so this issue is better brought up in a meeting with the original project partners. 

In terms of things I can do in the mean time, I could start with the shell of the application. As seen in outline of javascript code above(after ".filterDate"), Tropomi data comes in time series format, so building the base for a function that can take the perameters of a given location, and compare it to the most recent NO2 reading should be possible before I get the ERA5 wind calculation finished. 

I also have other test sites picked out for when I do get a working function. In the interest of keeping variables to a minimum, I am continuing along the theme of coal plants in Colorado. The Comanche Generating Station in Pueblo is the first one to compare, as it fits all the aforementioned variables, as well as has interesting Tropomi results. A third site could be the Martin Drake Power Station in Colorado springs, with less interesting Tropomi results, but nonetheless useful in testing if my function works. 

Overall, I am seeing similar patterns for what I was initially warned about- strange pixel values specifically around coal plants. So far, the data has supported this, which brings hope there is some correlation between Tropomi data and wind disruption to be found. 

# Unworking attempt at bringing in geotiff file

In [2]:
import os
import rasterio as rio
from rasterio.plot import plotting_extent

import earthpy as et
import earthpy.clip as cl
import earthpy.plot as ep
import earthpy.spatial as es

# os.chdir(os.path.join(et.io.HOME, "earth-analytics"))

# tropomi_path = os.path.join("data", "final_project", "studyarea2.tif")

# with rio.open(tropomi_path) as src: 
#     tropomi_data = src.read()
#     tropomi_meta = src.meta
    
# ep.plot_bands(tropomi_data)

## Citations 

* Richter, A., Burrows, J., Nüß, H. et al. Increase in tropospheric nitrogen dioxide over China observed from space. Nature 437, 129–132 (2005). https://doi.org/10.1038/nature04092

* G Bylin, G Hedenstierna, T Lindvall, B Sundin
European Respiratory Journal 1988 1: 606-612;